# Creating a dataset

This tutorial provides a short example on how to create a SeisBench dataset. Datasets can be created from any event catalog and waveform collection. For this example, we download an event catalog for Switzerland through FDSN. We will then download the associated waveforms through FDSN as well. We use built-in SeisBench functions for writing out the dataset in SeisBench format. In this example notebook we aim for an easy example outlining the principles of dataset creation. There are a few further considerations, in particular, for converting larger datasets, that we outline at the end.

**Note:** Some familiarity with obspy and its FDSN client is helpful for this tutorial, but not required.

In [1]:
import seisbench
import seisbench.data as sbd
import seisbench.util as sbu

import obspy
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.header import FDSNException
import matplotlib.pyplot as plt
from pathlib import Path
import pickle
import os

with open(os.path.join('{0}/{1}'.format(seisbench.cache_root,'datasets/chile/files_paths'), 'day1.pkl'),'rb') as fp:
    DF_picks = pickle.load(fp)

DF_picks.shape

In [22]:
first_day = '/home/javak/Sample_data_chile/Updated Catalog from Crist/catalog_2007_2021/manual picks/day1.xml'
client = Client("GFZ", timeout=10)
catalog_day_1 = obspy.core.event.read_events(pathname_or_url=first_day, format=None)
print(catalog_day_1.__str__(print_all=True))

125 Event(s) in Catalog:
2012-06-30T00:08:58.997298Z | -21.613,  -68.316
2012-06-30T00:13:03.397439Z | -21.606,  -68.369
2012-06-30T00:16:50.550993Z | -21.337,  -68.838
2012-06-30T00:21:36.703147Z | -24.702,  -67.118
2012-06-30T00:23:59.443037Z | -20.148,  -69.042
2012-06-30T00:29:58.951745Z | -19.396,  -69.230
2012-06-30T01:33:37.744907Z | -21.222,  -70.222
2012-06-30T01:37:25.038130Z | -20.775,  -70.573
2012-06-30T02:08:47.142973Z | -23.032,  -70.710
2012-06-30T02:10:42.348289Z | -22.369,  -68.493
2012-06-30T02:13:45.020455Z | -22.023,  -68.581
2012-06-30T02:17:27.486495Z | -21.813,  -68.406
2012-06-30T02:25:54.917135Z | -21.757,  -68.467
2012-06-30T02:28:27.505323Z | -15.845,  -72.000
2012-06-30T02:31:59.154528Z | -23.814,  -70.371
2012-06-30T02:36:47.707373Z | -20.777,  -68.655
2012-06-30T02:50:04.966915Z | -21.136,  -68.608
2012-06-30T02:54:17.542345Z | -21.111,  -68.736
2012-06-30T03:00:15.646496Z | -21.166,  -68.680
2012-06-30T03:02:36.338047Z | -21.393,  -68.461
2012-06-30T03:1

In [23]:
second_day = '/home/javak/Sample_data_chile/Updated Catalog from Crist/catalog_2007_2021/manual picks/day2.xml'
client = Client("GFZ", timeout=10)
catalog_day_2 = obspy.core.event.read_events(pathname_or_url=second_day, format=None)
print(catalog_day_2.__str__(print_all=True))

In [28]:
catalog = catalog_day_1.__add__(catalog_day_2)
catalog.write("/home/javak/Sample_data_chile/Updated Catalog from Crist/catalog_2007_2021/manual picks/catalog_2012_2018.xml", format="QUAKEML")  
print(catalog.__str__(print_all=True))

251 Event(s) in Catalog:
2012-06-30T00:08:58.997298Z | -21.613,  -68.316
2012-06-30T00:13:03.397439Z | -21.606,  -68.369
2012-06-30T00:16:50.550993Z | -21.337,  -68.838
2012-06-30T00:21:36.703147Z | -24.702,  -67.118
2012-06-30T00:23:59.443037Z | -20.148,  -69.042
2012-06-30T00:29:58.951745Z | -19.396,  -69.230
2012-06-30T01:33:37.744907Z | -21.222,  -70.222
2012-06-30T01:37:25.038130Z | -20.775,  -70.573
2012-06-30T02:08:47.142973Z | -23.032,  -70.710
2012-06-30T02:10:42.348289Z | -22.369,  -68.493
2012-06-30T02:13:45.020455Z | -22.023,  -68.581
2012-06-30T02:17:27.486495Z | -21.813,  -68.406
2012-06-30T02:25:54.917135Z | -21.757,  -68.467
2012-06-30T02:28:27.505323Z | -15.845,  -72.000
2012-06-30T02:31:59.154528Z | -23.814,  -70.371
2012-06-30T02:36:47.707373Z | -20.777,  -68.655
2012-06-30T02:50:04.966915Z | -21.136,  -68.608
2012-06-30T02:54:17.542345Z | -21.111,  -68.736
2012-06-30T03:00:15.646496Z | -21.166,  -68.680
2012-06-30T03:02:36.338047Z | -21.393,  -68.461
2012-06-30T03:1

In [ ]:
def get_event_params(event):
    origin = event.origins[0]
    #mag = event.preferred_magnitude()

    source_id = str(event.origins[0].resource_id)

    event_params = {
        "source_id": source_id,
        "source_origin_time": str(origin.time),
        "source_origin_uncertainty_sec": origin.time_errors["uncertainty"],
        "source_latitude_deg": origin.latitude,
        "source_latitude_uncertainty_km": origin.latitude_errors["uncertainty"],
        "source_longitude_deg": origin.longitude,
        "source_longitude_uncertainty_km": origin.longitude_errors["uncertainty"],
        "source_depth_km": origin.depth / 1e3,
        "source_depth_uncertainty_km": origin.depth_errors["uncertainty"] / 1e3,
    }

    #if mag is not None:
    #    event_params["source_magnitude"] = mag.mag
    #    event_params["source_magnitude_uncertainty"] = mag.mag_errors["uncertainty"]
    #    event_params["source_magnitude_type"] = mag.magnitude_type
    #    event_params["source_magnitude_author"] = mag.creation_info.agency_id
    
    #if str(origin.time) < "2015-01-07":
    #    split = "train"
    #elif str(origin.time) < "2015-01-08":
    #    split = "dev"
    #else:
    #    split = "test"
    split = 'train'
    event_params["split"] = split
    
    return event_params

get_event_params(catalog[0])

{'source_id': 'smi:de.erdbeben-in-bayern/origin/1f49ac14-fc20-4284-b859-ce5095590226',
 'source_origin_time': '2012-06-30T00:08:58.997298Z',
 'source_origin_uncertainty_sec': None,
 'source_latitude_deg': -21.613279,
 'source_latitude_uncertainty_km': None,
 'source_longitude_deg': -68.316424,
 'source_longitude_uncertainty_km': None,
 'source_depth_km': 150.517578,
 'source_depth_uncertainty_km': 7.74228562586,
 'split': 'train'}

In [ ]:
def get_trace_params(pick):
    net = pick.waveform_id.network_code
    sta = pick.waveform_id.station_code

    trace_params = {
        "station_network_code": net,
        "station_code": sta,
        "trace_channel": pick.waveform_id.channel_code[:2],
        "station_location_code": pick.waveform_id.location_code,
    }

    return trace_params

get_trace_params(catalog[12].picks[0])

{'station_network_code': 'CX',
 'station_code': 'PB01',
 'trace_channel': 'HH',
 'station_location_code': ''}

In [ ]:
def get_waveforms(pick, trace_params, time_before=60, time_after=60):
    t_start = pick.time - time_before
    t_end = pick.time + time_after
    
    try:
        waveforms = client.get_waveforms(
            network=trace_params["station_network_code"],
            station=trace_params["station_code"],
            location="*",
            channel=f"{trace_params['trace_channel']}*",
            starttime=t_start,
            endtime=t_end,
        )
    except FDSNException:
        # Return empty stream
        waveforms = obspy.Stream()
    
    return waveforms
    
pick = catalog[1].picks[5]
trace_params = get_trace_params(pick)
get_waveforms(pick, trace_params)

3 Trace(s) in Stream:
CX.PB06..HHZ | 2012-06-30T00:12:35.468393Z - 2012-06-30T00:14:35.468393Z | 100.0 Hz, 12001 samples
CX.PB06..HHE | 2012-06-30T00:12:35.468393Z - 2012-06-30T00:14:35.468393Z | 100.0 Hz, 12001 samples
CX.PB06..HHN | 2012-06-30T00:12:35.468394Z - 2012-06-30T00:14:35.468394Z | 100.0 Hz, 12001 samples

In [2]:
base_path = Path(".")
metadata_path = base_path / "metadata.csv"
waveforms_path = base_path / "waveforms.hdf5"

In [27]:
# Iterate over events and picks, write to SeisBench format
with sbd.WaveformDataWriter(metadata_path, waveforms_path) as writer:
    
    # Define data format
    writer.data_format = {
        "dimension_order": "CW",
        "component_order": "ZNE",
        "measurement": "velocity",
        "unit": "counts",
        "instrument_response": "not restituted",
    }
    
    for event in catalog:
        event_params = get_event_params(event)
        for pick in event.picks:
            trace_params = get_trace_params(pick)
            waveforms = get_waveforms(pick, trace_params)
            
            if len(waveforms) == 0:
                # No waveform data available
                continue
        
            sampling_rate = waveforms[0].stats.sampling_rate
            # Check that the traces have the same sampling rate
            assert all(trace.stats.sampling_rate == sampling_rate for trace in waveforms)
            
            actual_t_start, data, _ = sbu.stream_to_array(
                waveforms,
                component_order=writer.data_format["component_order"],
            )
            
            trace_params["trace_sampling_rate_hz"] = sampling_rate
            trace_params["trace_start_time"] = str(actual_t_start)
            
            sample = (pick.time - actual_t_start) * sampling_rate
            trace_params[f"trace_{pick.phase_hint}_arrival_sample"] = int(sample)
            trace_params[f"trace_{pick.phase_hint}_status"] = pick.evaluation_mode
            
            writer.add_trace({**event_params, **trace_params}, data)

Traces converted: 0it [00:00, ?it/s]

In [3]:
data = sbd.WaveformDataset(base_path, sampling_rate=100)

In [4]:
print("Training examples:", len(data.train()))
print("Development examples:", len(data.dev()))
print("Test examples:", len(data.test()))

Training examples: 2053
Development examples: 341
Test examples: 285


In [ ]:
data.metadata

,source_id,source_origin_time,source_origin_uncertainty_sec,source_latitude_deg,source_latitude_uncertainty_km,source_longitude_deg,source_longitude_uncertainty_km,source_depth_km,source_depth_uncertainty_km,split,...,station_location_code,trace_sampling_rate_hz,trace_start_time,trace_P_arrival_sample,trace_P_status,trace_name,trace_S_arrival_sample,trace_S_status,trace_chunk,trace_component_order
0,smi:de.erdbeben-in-bayern/origin/1f49ac14-fc20...,2012-06-30T00:08:58.997298Z,NaN,-21.613279,NaN,-68.316424,NaN,150.517578,7.742286,train,...,NaN,100.0,2012-06-30T00:08:26.048391Z,6000.0,NaN,"bucket0$0,:3,:12001",NaN,NaN,_day1,ZNE
1,smi:de.erdbeben-in-bayern/origin/1f49ac14-fc20...,2012-06-30T00:08:58.997298Z,NaN,-21.613279,NaN,-68.316424,NaN,150.517578,7.742286,train,...,NaN,100.0,2012-06-30T00:08:29.038393Z,6000.0,NaN,"bucket0$1,:3,:12001",NaN,NaN,_day1,ZNE
2,smi:de.erdbeben-in-bayern/origin/1f49ac14-fc20...,2012-06-30T00:08:58.997298Z,NaN,-21.613279,NaN,-68.316424,NaN,150.517578,7.742286,train,...,NaN,100.0,2012-06-30T00:08:27.798393Z,6000.0,NaN,"bucket0$2,:3,:12001",NaN,NaN,_day1,ZNE
3,smi:de.erdbeben-in-bayern/origin/1f49ac14-fc20...,2012-06-30T00:08:58.997298Z,NaN,-21.613279,NaN,-68.316424,NaN,150.517578,7.742286,train,...,NaN,100.0,2012-06-30T00:08:36.608391Z,6000.0,NaN,"bucket0$3,:3,:12001",NaN,NaN,_day1,ZNE
4,smi:de.erdbeben-in-bayern/origin/1f49ac14-fc20...,2012-06-30T00:08:58.997298Z,NaN,-21.613279,NaN,-68.316424,NaN,150.517578,7.742286,train,...,NaN,100.0,2012-06-30T00:08:30.298393Z,6000.0,NaN,"bucket0$4,:3,:12001",NaN,NaN,_day1,ZNE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916,smi:de.erdbeben-in-bayern/origin/8e8a06e7-50df...,2012-06-30T23:30:29.716330Z,NaN,-20.983631,NaN,-67.178248,NaN,197.220703,6.512044,train,...,NaN,100.0,2012-06-30T23:30:36.138393Z,NaN,NaN,"bucket1$849,:3,:12001",6000.0,NaN,_day1,ZNE
1917,smi:de.erdbeben-in-bayern/origin/8e8a06e7-50df...,2012-06-30T23:30:29.716330Z,NaN,-20.983631,NaN,-67.178248,NaN,197.220703,6.512044,train,...,NaN,100.0,2012-06-30T23:30:43.808393Z,NaN,NaN,"bucket1$850,:3,:12001",6000.0,NaN,_day1,ZNE
1918,smi:de.erdbeben-in-bayern/origin/8e8a06e7-50df...,2012-06-30T23:30:29.716330Z,NaN,-20.983631,NaN,-67.178248,NaN,197.220703,6.512044,train,...,NaN,100.0,2012-06-30T23:30:46.388393Z,NaN,NaN,"bucket1$851,:3,:12001",6000.0,NaN,_day1,ZNE
1919,smi:de.erdbeben-in-bayern/origin/8e8a06e7-50df...,2012-06-30T23:30:29.716330Z,NaN,-20.983631,NaN,-67.178248,NaN,197.220703,6.512044,train,...,NaN,100.0,2012-06-30T23:30:45.938393Z,NaN,NaN,"bucket1$852,:3,:12001",6000.0,NaN,_day1,ZNE


## Considerations for converting datasets

As outlined above, this tutorial provides a very minimal example on converting a dataset. Here we outline additional consideration that should be taken into account when preparing a dataset.

- **Grouping picks**: In this example, we created one trace for each pick. Naturally, traces will overlap if multiple picks, e.g., P and S phases, are available for an event at a station. For an example implementation of this grouping operation, have a look [here](https://github.com/seisbench/seisbench/blob/df94dcd86ce66d6a2ee2bd00da3857259fe579bd/seisbench/data/ethz.py#L109) and in the subsequent lines.
- **Adding station information**: In this example, we added no station information except its name. In practice, it will often be helpful for users to incorporate, for example, the location of the station. We skipped this step here, because it requires loading station inventories through FDSN. For an example implementation, have a look [here](https://github.com/seisbench/seisbench/blob/df94dcd86ce66d6a2ee2bd00da3857259fe579bd/seisbench/data/ethz.py#L315).
- **Memory requirements**: Internally, the `WaveformDataWriter` writes out the the waveforms continuously in blocks (see point below), but keeps all metadata in memory until the dataset is complete. For very large datasets (or very detailed metadata) this can result in several gigabytes of memory consumption. If you are writing such datasets, make sure the available memory on your machine is sufficient.
- **Waveform blocks**: Instead of writing each waveform separately, waveforms are written out in blocks. This massively improves IO performance. Have a look at [the documentation](https://seisbench.readthedocs.io/en/stable/pages/data_format.html#traces-blocks) for details on the strategy. We expect that in nearly all cases using the default setting will be a good choice.
- **FDSN considerations**: When converting very large datasets, the performance might be limited by the performance of the FDSN webservice. Unfortunately, downloading lots of short waveforms (as required for many machine learning applications) does not seem to be the most favorable use case for FDSN. This leads to rather slow performance when naively downloading the waveforms as outlined above. Instead, it is often helpful to issue [bulk requests](https://docs.obspy.org/master/packages/autogen/obspy.clients.fdsn.client.Client.get_waveforms_bulk.html). In addition, it might be a good choice to first download the waveforms and cache them locally, for example, in .mseed format, and then convert them to SeisBench.

For further details on the data format, check out [the data format specification in the SeisBench documentation](https://seisbench.readthedocs.io/en/stable/pages/data_format.html#traces-blocks).